<a href="https://colab.research.google.com/github/FUJITOSHION/signate_compe/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime
import random
from tqdm import tqdm
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
def fix_seed(seed):
    np.random.seed(seed=seed)
    random.seed(seed)

seed=121
fix_seed(seed)

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/train/data/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/train/data/test.csv')
info_data = pd.read_csv('/content/drive/MyDrive/train/data/info.csv')
network = pd.read_csv('/content/drive/MyDrive/train/data/network.csv')

In [ ]:
train_data['target'] = 0
data = pd.concat([train_data, test_data], axis=0).reset_index().drop(['index'], axis=1)
# infoを結合（前処理が必要）

In [ ]:
def date_split(df:pd.DataFrame):
    df['date'] = df['date'].astype(str)
    df['year'] = df['date'].apply(lambda x: x[:4]).astype(int)
    df['month'] = df['date'].apply(lambda x: x[4:6]).astype(int)
    df['day'] = df['date'].apply(lambda x: x[6:8]).astype(int)
    data['hour'] = data['planArrival'].apply(lambda x: x[:2])
    data['second'] = data['planArrival'].apply(lambda x: x[3:])
    data['hour'] = data['hour'].apply(lambda x: int(x[1]) if '0' in x[0] else int(x))
    data['second'] = data['second'].apply(lambda x: int(x[1]) if '0' in x[0] else int(x))
    return df

In [ ]:
def make_lag(df, lag):
    name = 'lag_' + str(lag)
    data[name] = data['delayTime'].shift(lag)

    for i in range(len(df)):
        if i - lag <0:
            continue
        be_train = data.at[i-lag, 'trainNo']
        now_train = data.at[i, 'trainNo']

        if be_train != now_train:
            data.at[i, name] = np.nan

    return df

In [ ]:
# time split
data = date_split(data)

# make tag
max_lag = 5
for i in tqdm(range(1, max_lag)):
    data = make_lag(data, i)

# データ分割
train_len = len(train_data)
test = data[train_len :]
train = data[: train_len]


100%|██████████| 4/4 [05:13<00:00, 78.37s/it]


In [ ]:
lc = LabelEncoder()
# data = pd.concat([train_data, test_data])
data['lineName'] = lc.fit_transform(data['lineName'])
data['directionCode'] = lc.fit_transform(data['directionCode'])
data['trainNo'] = lc.fit_transform(data['trainNo'])
data['stopStation'] = lc.fit_transform(data['stopStation'])
del data['planArrival']
del data['id']
del data['date']

In [ ]:
# データ分割
train_len = len(train_data)
test = data[train_len :]
train = data[: train_len]

In [ ]:
train = train.fillna(method='bfill')
del train['target']
train, val = train_test_split(train, random_state = seed)

In [ ]:
train_y = train['delayTime']
train_x = train.drop(['delayTime'], axis = 1)

val_y= val['delayTime']
val_x = val.drop(['delayTime'], axis = 1)
params = {
        'task' : 'train',
        'boosting':'gbdt',
        'objective' : 'regression',
        'metric' : {'mae'},
        'num_leaves':78,
        'drop_rate':0.05,
        'learning_rate':0.01,
        'seed':seed,
        'verbose':0,
        'device': 'cpu'
}
lgb_train = lgb.Dataset(train_x, train_y)
lgb_eval = lgb.Dataset(val_x, val_y, reference=lgb_train)

evaluation_results = {}
lgbm = lgb.train(params,
                lgb_train,
                num_boost_round=3000,
                valid_sets=[lgb_train, lgb_eval],
                valid_names=['train', 'val'],
                evals_result=evaluation_results,
                early_stopping_rounds=300,
                verbose_eval=100)

In [ ]:
test_data = test.query('target == 1')
del test_data['delayTime']
del test_data['target']

In [ ]:
test_data = test_data.reset_index()

In [ ]:
del test_data['level_0']

In [ ]:
del test_data['index']

In [ ]:
test_data

,lineName,directionCode,trainNo,stopStation,year,month,day,hour,second,lag_1,lag_2,lag_3,lag_4
0,0,0,177,61,2019,12,1,18,2,3.000000,2.000000,3.000000,3.000000
1,0,0,177,34,2019,12,1,18,4,2.006346,3.000000,2.000000,3.000000
2,0,0,177,51,2019,12,1,18,6,3.531146,2.006346,3.000000,2.000000
3,0,0,177,39,2019,12,1,18,8,5.523942,3.531146,2.006346,3.000000
4,0,0,177,10,2019,12,1,18,10,6.997230,5.523942,3.531146,2.006346
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123270,3,1,2846,72,2020,10,21,19,52,48.337610,44.549852,48.142259,42.330878
123271,3,1,2846,44,2020,10,21,19,54,51.338708,48.337610,44.549852,48.142259
123272,3,1,2846,22,2020,10,21,19,56,NaN,51.338708,48.337610,44.549852
123273,3,1,2846,53,2020,10,21,19,59,NaN,NaN,51.338708,48.337610


In [ ]:
preds = []
for j in range(len(test_data)):
    if j+4 == 123274:
        break
    n_train = test_data.at[j, 'trainNo']
    a_train = test_data.at[j+1, 'trainNo']
    b_train = test_data.at[j+2, 'trainNo']
    c_train = test_data.at[j+3, 'trainNo']
    d_train = test_data.at[j+4, 'trainNo']

    pred = lgbm.predict(test_data.iloc[j,:])
    preds.append(pred)
    
    if n_train == a_train:
        test_data.at[j+1, 'lag_1'] = pred

        if n_train == b_train:
            test_data.at[j+2, 'lag_2'] = pred

            if n_train == c_train:
                test_data.at[j+3, 'lag_3'] = pred
                
                if n_train == d_train:
                    test_data.at[j+4, 'lag_4'] = pred


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:478: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [ ]:
test_data.tail()

,lineName,directionCode,trainNo,stopStation,year,month,day,hour,second,lag_1,lag_2,lag_3,lag_4
123270,3,1,2846,72,2020,10,21,19,52,1.134888,0.638524,1.080021,0.732004
123271,3,1,2846,44,2020,10,21,19,54,0.875991,1.134888,0.638524,1.080021
123272,3,1,2846,22,2020,10,21,19,56,NaN,0.875991,1.134888,0.638524
123273,3,1,2846,53,2020,10,21,19,59,NaN,NaN,0.875991,1.134888
123274,3,1,2969,77,2020,10,21,19,1,0.000000,0.000000,0.000000,0.000000


In [ ]:
pred = lgbm.predict(test_data.iloc[123270,:])
preds.append(pred)
pred = lgbm.predict(test_data.iloc[123271,:])
test_data.at[123272,'lag_1'] = pred
test_data.at[123273,'lag_2'] = pred
preds.append(pred)
pred = lgbm.predict(test_data.iloc[123272,:])
preds.append(pred)
test_data.at[123273,'lag_1'] = pred
pred = lgbm.predict(test_data.iloc[123273,:])
preds.append(pred)
pred = lgbm.predict(test_data.iloc[123274,:])
preds.append(pred)

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:478: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [ ]:
test_datas = pd.read_csv('/content/drive/MyDrive/train/data/test.csv')
test_id = test_datas.query('target == 1')
test_id = test_id['id']
a = [float(i) for i in preds]
submit = pd.DataFrame({'id':test_id, 'y':a})
submit['y'] = submit['y'].apply(lambda x: 0 if x < 0.9 else x)
submit['y'] = submit['y'].apply(lambda x: 0 if 0 > x else x)
now = datetime.datetime.now()
submit.to_csv('now'+str(now)+'.csv', index = None, header=None)

In [ ]:
print(sum(submit['y'])/len(submit), sum(train['delayTime'])/len(train))

1.256060106635205 1.2105397958023145
